# Predicción de pobreza usando Random Forest

Usaremos Random Forest para clasificar hogares entre pobre y no pobre. Se entiende como pobre un hogar cuyos miembros disponen de un ingreso percápita inferior a la línea de pobreza. Usaremos los datos de Colombia para 2024 y un conjunto pequeño de variables predictoras. La LP corresponde a la definición oficial. Esta es una tarea de aprendizaje supervisado

## Datos

Usamos los datos de la médición de pobreza monetaria para 2024, que se pueden descargar [acá](https://microdatos.dane.gov.co/index.php/catalog/874/data-dictionary/F5?file_name=Personas)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

In [ ]:
file_p= "C:\\Users\\andre\\OneDrive - Universidad del Norte\\Drive\\Uninorte\\datos pobreza monetaria\\Personas_24.csv"
personas = pd.read_csv(file_p)
print(personas.shape)

file_h="C:\\Users\\andre\\OneDrive - Universidad del Norte\\Drive\\Uninorte\\datos pobreza monetaria\\Hogares_24.csv"
hogares = pd.read_csv(file_h)
print(hogares.shape)

El análisis se hará a nivel de hogar. Identificamos cada hogar como pobre, 1, o no pobre, 0. Como variables predictoras se usarán la edad, género, nivel educativo, posición ocupacional y estátus laboral del jefe de hogar, además del tamaño del hogar. USaremos únicamente datos para la la clase cabecera. 

In [ ]:
# Seleccionar datos Hogares

columns_to_keep = ['directorio', 'secuencia_p', 'clase','npersug','pobre']
hogares_selected = hogares[columns_to_keep].copy()
hogares_selected.describe()


In [ ]:
# Filtrar por clase: cabecera
hogares_selected = hogares_selected[hogares_selected['clase'] == 1]
print(hogares_selected.shape)

In [ ]:
# Seleccionar datos Personas
columns_to_keep = ['directorio', 'secuencia_p','clase','p3271','p6040','p6050','p3042','p6430','oc','des','fft']
personas_selected = personas[columns_to_keep].copy()
personas_selected.describe()


In [ ]:
# Seleccionar jefe de hogar y cabecera
personas_selected = personas_selected[(personas_selected['clase'] == 1) & (personas_selected['p6050'] == 1)]
personas_selected=personas_selected.drop('clase',axis=1)
print(personas_selected.shape)

In [ ]:
# Unir Hogares y personas

df=pd.merge(hogares_selected,personas_selected,on=['directorio','secuencia_p'],how='left')
print(df.shape)
df.head()

In [ ]:
# Replace missing values in p6430 based on employment status
print("Before replacing missing values:")
print(f"Missing values in p6430: {df['p6430'].isna().sum()}")

# Check current distribution of employment status variables
print(f"\nDes==1 (unemployed): {(df['des']==1).sum()}")
print(f"Fft==1 (out of labor force): {(df['fft']==1).sum()}")

# Replace missing values based on conditions
# If des==1 (unemployed), replace missing p6430 with 9
df.loc[(df['p6430'].isna()) & (df['des'] == 1), 'p6430'] = 9

# If fft==1 (out of labor force), replace missing p6430 with 10
df.loc[(df['p6430'].isna()) & (df['fft'] == 1), 'p6430'] = 10

print("\nAfter replacing missing values:")
print(f"Missing values in p6430: {df['p6430'].isna().sum()}")

# Show the replacements made
replaced_des = ((df['p6430'] == 9) & (df['des'] == 1)).sum()
replaced_fft = ((df['p6430'] == 10) & (df['fft'] == 1)).sum()

print(f"\nReplacements made:")
print(f"Replaced with 9 (unemployed): {replaced_des}")
print(f"Replaced with 10 (out of labor force): {replaced_fft}")

# Show updated frequency table for p6430
print("\nUpdated frequency table for p6430:")
print("="*40)
freq_table = df['p6430'].value_counts().sort_index()
print(freq_table)
print("-" * 20)
print(f"Total: {freq_table.sum():,}")

In [ ]:
# Rename p6430 to est_lab (estatus laboral)
df = df.rename(columns={'p6430': 'est_lab'})
df=df.drop(['oc','des','fft'],axis=1)


## Árbol de regresión

Haremos un árbol de regresión y después lo podamos. Nuestra variable objetivo es **pobre**

Primero preparamos los datos y hacemos la partición de muestra para entrenamiento y prueba

In [ ]:
# Drop missing values and prepare data for modeling
print("Before dropping missing values:")
print(f"DataFrame shape: {df.shape}")
print(f"Missing values per column:")
print(df.isnull().sum())

# Drop rows with any missing values
df_clean = df.dropna()

print(f"\nAfter dropping missing values:")
print(f"DataFrame shape: {df_clean.shape}")
print(f"Rows dropped: {len(df) - len(df_clean)}")


In [ ]:

# Define features (X) and target variable (y)
# Features: age, age^2, gender, education, labor status, household size
feature_columns = ['p6040', 'p3271', 'p3042', 'est_lab', 'npersug']  
X = df_clean[feature_columns].copy()

# Create age squared
X['edad_cuadrado'] = X['p6040'] ** 2

# Target variable: poverty status
y = df_clean['pobre'].copy()

print(f"\nFinal data for modeling:")
print(f"Features (X) shape: {X.shape}")
print(f"Target (y) shape: {y.shape}")
print(f"Feature columns: {list(X.columns)}")

# Check target variable distribution
print(f"\nTarget variable distribution:")
print(y.value_counts())
print(f"Poverty rate: {y.mean():.2%}")

Las variables p2371, y est_lab son categóricas no ordinales. Usaremos one hot encoding para ingresarlas adecuadamente al modelo. Para p3042 usamos label encoding pues debemos mantener el orden de los niveles educativos. 

In [ ]:
# Encoding categorical variables
from sklearn.preprocessing import LabelEncoder

print("Before encoding:")
print(f"X shape: {X.shape}")
print(f"Unique values in p3271 (gender): {sorted(X['p3271'].unique())}")
print(f"Unique values in est_lab (labor status): {sorted(X['est_lab'].unique())}")
print(f"Unique values in p3042 (education): {sorted(X['p3042'].unique())}")

# 1. One-hot encoding for categorical non-ordinal variables
# p3271 (gender) and est_lab (labor status)
X_encoded = pd.get_dummies(X, columns=['p3271', 'est_lab'], prefix=['gender', 'lab_status'])

# 2. Label encoding for ordinal variable p3042 (education level)
# Keep the order: higher numbers = higher education
label_encoder = LabelEncoder()
X_encoded['education_level'] = label_encoder.fit_transform(X['p3042'])

# Drop the original p3042 column since we have the encoded version
X_encoded = X_encoded.drop('p3042', axis=1)

print(f"\nAfter encoding:")
print(f"X_encoded shape: {X_encoded.shape}")
print(f"New columns: {list(X_encoded.columns)}")

# Show the mapping for education levels
education_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(f"\nEducation level mapping: {education_mapping}")

# Update X for modeling
X = X_encoded.copy()

print(f"\nFinal feature set:")
print(f"Shape: {X.shape}")
print(f"Columns: {list(X.columns)}")

# Check for any remaining missing values
print(f"\nMissing values check:")
print(X.isnull().sum().sum())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 

In [ ]:
clpobre=DecisionTreeClassifier(random_state=42, max_depth=3)
clpobre.fit(X_train,y_train)
y_pred=clpobre.predict(X_test)
accuracy_score(y_test,y_pred)
print(f"Accuracy: {accuracy_score(y_test,y_pred)}")

# Enhanced plot with better readability
plt.figure(figsize=(20,12))  # Increased figure size
plot_tree(clpobre, 
          feature_names=X.columns, 
          class_names=['Not Poor', 'Poor'], 
          filled=True,
          fontsize=12,        # Increase font size
          proportion=False,   # Show actual counts instead of proportions
          impurity=True,      # Show impurity measure
          rounded=True,       # Rounded boxes
          precision=2)        # Decimal precision for values
plt.title("Decision Tree for Poverty Classification", fontsize=16, pad=20)
plt.tight_layout()  # Adjust layout to prevent clipping
plt.show()


### Interpretación de los valores en cada nodo del árbol

Cada caja (nodo) del árbol de decisión contiene la siguiente información:

#### **1. Condición de división (solo en nodos internos)**
- Ejemplo: `p6040 <= 35.5`
- **Significado**: La edad del jefe de hogar es menor o igual a 35.5 años
- **Decisión**: Si es verdadero → rama izquierda, Si es falso → rama derecha

#### **2. Gini Impurity (impureza)**
- Ejemplo: `gini = 0.42`
- **Rango**: 0 a 0.5 (para clasificación binaria)
- **Significado**: 
  - `gini = 0`: Nodo puro (todas las observaciones pertenecen a la misma clase)
  - `gini = 0.5`: Máxima impureza (50% pobres, 50% no pobres)
  - **Fórmula**: gini = 1 - (p₀² + p₁²), donde p₀ y p₁ son las proporciones de cada clase

#### **3. Samples (muestras)**
- Ejemplo: `samples = 1,234`
- **Significado**: Número total de observaciones (hogares) que llegan a este nodo
- **Importante**: La suma debe coincidir con el tamaño de la muestra de entrenamiento

#### **4. Value (distribución de clases)**
- Ejemplo: `value = [800, 434]`
- **Formato**: [No pobres, Pobres]
- **Significado**: 
  - 800 hogares clasificados como "No pobres" (clase 0)
  - 434 hogares clasificados como "Pobres" (clase 1)
  - **Verificación**: 800 + 434 = 1,234 (debe coincidir con samples)

#### **5. Class (predicción del nodo)**
- Ejemplo: `class = Not Poor`
- **Significado**: Clase mayoritaria en este nodo
- **Decisión**: Si una observación llega a este nodo, se clasifica con esta etiqueta
- **Regla**: La clase con mayor valor en el array `value`

#### **6. Color del nodo**
- **Naranja claro/oscuro**: Mayor proporción de pobres
- **Azul claro/oscuro**: Mayor proporción de no pobres  
- **Intensidad del color**: Indica qué tan "pura" es la clasificación

#### **Ejemplo de interpretación completa:**
```
est_lab <= 4.5
gini = 0.45
samples = 10,000
value = [6,000, 4,000]
class = Not Poor
```

**Interpretación**: "En este nodo, si el estatus laboral es ≤ 4.5, tenemos 10,000 hogares, de los cuales 6,000 son no pobres y 4,000 son pobres. La impureza es alta (0.45), pero como hay más no pobres, la predicción es 'Not Poor'."

### Poda del árbol

Hicimos un árbol pequeño, pero esa fue una decisión caprichosa. Pueden haber mejores árboles. Investigue sobre técnicas de poda. Una de ellas es minimal cost-complexity pruning. Investigue sobre ella y haga una implementación

### Comparación

Usando el mismo conjunto de variables predictoras y las transformaciones que considere adecuadas, ajuste un modelo logit de predicción de la pobreza. Compare con los resultados del árbol de decisión